In [0]:
%sql
USE CATALOG `nokia-assginment-catalog`;
-- drop schema patent_data cascade;

In [0]:
# from pyspark.sql import SparkSession
# from delta.tables import DeltaTable
# import os
# import time
# from datetime import datetime
# import traceback
# import json

# def initialize_spark():
#     """Initialize Spark session with Delta Lake support"""
#     return SparkSession.builder \
#         .appName("Patent Delta Tables Registration") \
#         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#         .getOrCreate()

# def check_path_exists(path):
#     """Check if a path exists and is accessible"""
#     try:
#         dbutils.fs.ls(path)
#         return True
#     except:
#         return False

# def is_delta_table(spark, path):
#     """Check if a path is a valid Delta table"""
#     try:
#         DeltaTable.forPath(spark, path)
#         return True
#     except:
#         return False

# def register_delta_table(spark, delta_path, table_name, database_name="patent_data", drop_if_exists=False):
#     """
#     Register a Delta table in the Databricks metastore
    
#     Args:
#         spark: SparkSession
#         delta_path: Path to Delta table
#         table_name: Name of the table to create
#         database_name: Database to create the table in
#         drop_if_exists: Whether to drop the table if it already exists
    
#     Returns:
#         bool: Success status
#     """
#     try:
#         # Create database if it doesn't exist
#         spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        
#         # Fully qualified table name
#         full_table_name = f"{database_name}.{table_name}"
        
#         # Check if table already exists
#         table_exists = False
#         existing_tables = [t.name for t in spark.catalog.listTables(database_name)]
#         if table_name in existing_tables:
#             table_exists = True
            
#         # Drop table if requested and it exists
#         if table_exists and drop_if_exists:
#             print(f"Dropping existing table {full_table_name}")
#             spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
#             table_exists = False
            
#         # Register Delta table
#         if not table_exists:
#             print(f"Creating table {full_table_name} from {delta_path}")
            
#             # Read the Delta table to get the data
#             delta_df = spark.read.format("delta").load(delta_path)
            
#             # Write directly to a table in the patent_data schema
#             delta_df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
            
#             # Configure table properties for optimization
#             spark.sql(f"""
#                 ALTER TABLE {full_table_name}
#                 SET TBLPROPERTIES (
#                     'delta.autoOptimize.optimizeWrite' = 'true',
#                     'delta.autoOptimize.autoCompact' = 'true',
#                     'delta.enableChangeDataFeed' = 'true',
#                     'delta.logRetentionDuration' = '30 days',
#                     'delta.tuneFileSizesForRewrites' = 'true'
#                 )
#             """)
                
#             print(f"Table {full_table_name} created with optimization properties")
#             return True
#         else:
#             print(f"Table {full_table_name} already exists")
#             return True
            
#     except Exception as e:
#         print(f"Error registering Delta table {table_name}: {str(e)}")
#         print(traceback.format_exc())
#         return False

# def perform_upsert(spark, table_name, source_path, database_name="patent_data"):
#     """
#     Perform upsert operation (merge) on a Delta table using PySpark
    
#     Args:
#         spark: SparkSession
#         table_name: Name of the table to update
#         source_path: Path to source Delta table with new data
#         database_name: Database containing the table
    
#     Returns:
#         dict: Result with status and message
#     """
#     try:
#         # Fully qualified table name
#         full_table_name = f"{database_name}.{table_name}"
        
#         # Read the new data
#         new_data_df = spark.read.format("delta").load(source_path)
        
#         # Check if publication_number exists in the source data
#         if "publication_number" not in new_data_df.columns:
#             return {
#                 "status": "error", 
#                 "message": f"Source data for {table_name} does not have publication_number column"
#             }
        
#         # Get target table
#         try:
#             target_table = DeltaTable.forName(spark, full_table_name)
#         except Exception as e:
#             return {
#                 "status": "error",
#                 "message": f"Could not access target table {full_table_name}: {str(e)}"
#             }
        
#         # Perform merge operation
#         try:
#             target_table.alias("target").merge(
#                 new_data_df.alias("source"),
#                 "target.publication_number = source.publication_number"
#             ).whenMatchedUpdateAll(
#             ).whenNotMatchedInsertAll(
#             ).execute()
            
#             return {
#                 "status": "success",
#                 "message": f"Upsert completed for {table_name}"
#             }
#         except Exception as e:
#             return {
#                 "status": "error",
#                 "message": f"Error during merge operation for {table_name}: {str(e)}"
#             }
            
#     except Exception as e:
#         return {
#             "status": "error",
#             "message": f"Error processing upsert for {table_name}: {str(e)}"
#         }

# def register_all_patent_tables(gold_base_path="/Volumes/nokia-assginment-catalog/gold", 
#                               database_name="patent_data",
#                               drop_existing=False):
#     """
#     Register all patent Delta tables in the Databricks metastore
    
#     Args:
#         gold_base_path: Base path to gold Delta tables
#         database_name: Database to create tables in
#         drop_existing: Whether to drop existing tables
        
#     Returns:
#         dict: Results with status for each table
#     """
#     spark = initialize_spark()
#     results = {}
    
#     # Define tables to register
#     tables = [
#         {"path": f"{gold_base_path}/patents", "name": "patents"},
#         {"path": f"{gold_base_path}/ipc_classifications", "name": "ipc_classifications"},
#         {"path": f"{gold_base_path}/ipc_individual", "name": "ipc_individual"},
#         {"path": f"{gold_base_path}/inventors", "name": "inventors"},
#         {"path": f"{gold_base_path}/inventors_individual", "name": "inventors_individual"},
#         {"path": f"{gold_base_path}/applicants", "name": "applicants"},
#         {"path": f"{gold_base_path}/applicants_individual", "name": "applicants_individual"},
#         {"path": f"{gold_base_path}/us_applicants", "name": "us_applicants"},
#         {"path": f"{gold_base_path}/us_applicants_individual", "name": "us_applicants_individual"},
#         {"path": f"{gold_base_path}/claims", "name": "claims"},
#         {"path": f"{gold_base_path}/claims_individual", "name": "claims_individual"},
#         {"path": f"{gold_base_path}/complete_patents", "name": "complete_patents"}
#     ]
    
#     # Register each table
#     for table_info in tables:
#         delta_path = table_info["path"]
#         table_name = table_info["name"]
        
#         # Check if Delta path exists
#         if not check_path_exists(delta_path):
#             print(f"Delta path does not exist: {delta_path}")
#             results[table_name] = "Path not found"
#             continue
            
#         # Check if it's a valid Delta table
#         if not is_delta_table(spark, delta_path):
#             print(f"Path is not a valid Delta table: {delta_path}")
#             results[table_name] = "Not a valid Delta table"
#             continue
            
#         # Register the table
#         success = register_delta_table(
#             spark, 
#             delta_path, 
#             table_name, 
#             database_name=database_name, 
#             drop_if_exists=drop_existing
#         )
        
#         if success:
#             results[table_name] = "Registered successfully"
#         else:
#             results[table_name] = "Registration failed"
    
#     return results

# def create_upsert_function(spark, database_name="patent_data"):
#     """
#     Create a SQL function for upserting data into tables
    
#     Args:
#         spark: SparkSession
#         database_name: Database containing the tables
#     """
#     try:
#         # For compatibility with your Databricks environment, we'll use a simple table function
#         # instead of a stored procedure
#         udf_code = f"""
#         CREATE OR REPLACE FUNCTION {database_name}.patent_upsert(table_name STRING, source_path STRING)
#         RETURNS STRING
#         LANGUAGE PYTHON
#         AS
#         $$
#         from delta.tables import DeltaTable
#         import json
        
#         try:
#             # Fully qualified table name
#             full_table_name = f"{database_name}.{{table_name}}"
            
#             # Read the new data
#             new_data_df = spark.read.format("delta").load(source_path)
            
#             # Check if publication_number exists in the source data
#             if "publication_number" not in new_data_df.columns:
#                 return json.dumps({{
#                     "status": "error", 
#                     "message": f"Source data does not have publication_number column"
#                 }})
            
#             # Get target table
#             try:
#                 target_table = DeltaTable.forName(spark, full_table_name)
#             except Exception as e:
#                 return json.dumps({{
#                     "status": "error",
#                     "message": f"Could not access target table {{full_table_name}}: {{str(e)}}"
#                 }})
            
#             # Perform merge operation
#             target_table.alias("target").merge(
#                 new_data_df.alias("source"),
#                 "target.publication_number = source.publication_number"
#             ).whenMatchedUpdateAll(
#             ).whenNotMatchedInsertAll(
#             ).execute()
            
#             return json.dumps({{
#                 "status": "success",
#                 "message": f"Upsert completed for {{table_name}}"
#             }})
            
#         except Exception as e:
#             return json.dumps({{
#                 "status": "error",
#                 "message": f"Error: {{str(e)}}"
#             }})
#         $$
#         """
        
#         spark.sql(udf_code)
#         print(f"Created upsert function {database_name}.patent_upsert")
#         return True
#     except Exception as e:
#         print(f"Error creating upsert function: {str(e)}")
#         print(traceback.format_exc())
#         return False

# def batch_upsert_function(spark, database_name="patent_data"):
#     """
#     Create a function to perform batch upserts across all tables
    
#     Args:
#         spark: SparkSession
#         database_name: Database containing the tables
#     """
#     try:
#         batch_function_code = f"""
#         CREATE OR REPLACE FUNCTION {database_name}.patent_upsert_job(base_path STRING)
#         RETURNS STRING
#         LANGUAGE PYTHON
#         AS
#         $$
#         import json
#         from delta.tables import DeltaTable
        
#         results = {{}}
        
#         # Tables to process
#         tables = [
#             "patents",
#             "ipc_classifications",
#             "ipc_individual",
#             "inventors",
#             "inventors_individual",
#             "applicants",
#             "applicants_individual",
#             "us_applicants",
#             "us_applicants_individual",
#             "claims",
#             "claims_individual",
#             "complete_patents"
#         ]
        
#         # Process each table
#         for table_name in tables:
#             try:
#                 # Paths
#                 source_path = f"{{base_path}}/{{table_name}}"
#                 full_table_name = f"{database_name}.{{table_name}}"
                
#                 # Check if path exists
#                 try:
#                     dbutils.fs.ls(source_path)
#                 except:
#                     results[table_name] = {{"status": "skipped", "message": "Source path not found"}}
#                     continue
                
#                 # Check if it's a valid Delta table
#                 try:
#                     source_df = spark.read.format("delta").load(source_path)
#                 except:
#                     results[table_name] = {{"status": "error", "message": "Not a valid Delta source"}}
#                     continue
                
#                 # Check if target table exists
#                 try:
#                     target_table = DeltaTable.forName(spark, full_table_name)
#                 except:
#                     results[table_name] = {{"status": "error", "message": "Target table not found"}}
#                     continue
                
#                 # Check for primary key
#                 if "publication_number" not in source_df.columns:
#                     results[table_name] = {{"status": "error", "message": "Source missing publication_number"}}
#                     continue
                
#                 # Perform merge operation
#                 target_table.alias("target").merge(
#                     source_df.alias("source"),
#                     "target.publication_number = source.publication_number"
#                 ).whenMatchedUpdateAll(
#                 ).whenNotMatchedInsertAll(
#                 ).execute()
                
#                 results[table_name] = {{"status": "success", "message": "Upsert completed"}}
                
#             except Exception as e:
#                 results[table_name] = {{"status": "error", "message": str(e)}}
        
#         return json.dumps(results)
#         $$
#         """
        
#         spark.sql(batch_function_code)
#         print(f"Created batch upsert function {database_name}.patent_upsert_job")
#         return True
#     except Exception as e:
#         print(f"Error creating batch upsert function: {str(e)}")
#         print(traceback.format_exc())
#         return False

# def create_control_table(spark, database_name="patent_data"):
#     """Create a control table for tracking updates"""
#     try:
#         spark.sql(f"""
#         CREATE TABLE IF NOT EXISTS {database_name}.patent_update_control (
#             update_id STRING,
#             update_timestamp TIMESTAMP,
#             source_path STRING,
#             result STRING,
#             PRIMARY KEY (update_id)
#         ) USING DELTA
#         """)
#         print(f"Created control table {database_name}.patent_update_control")
#         return True
#     except Exception as e:
#         print(f"Error creating control table: {str(e)}")
#         print(traceback.format_exc())
#         return False

# def setup_upsert_usage_instructions():
#     """
#     Generate instructions for using the upsert capabilities
#     """
#     instructions = """
#     -- To upsert data for a single table:
#     SELECT patent_data.patent_upsert('patents', '/path/to/new/patents/data');
    
#     -- To batch upsert data for all tables from a base directory:
#     SELECT patent_data.patent_upsert_job('/path/to/new/data');
    
#     -- To track updates in the control table:
#     INSERT INTO patent_data.patent_update_control
#     VALUES (
#         uuid(), -- Generate a unique ID
#         current_timestamp(),
#         '/path/to/processed/data',
#         patent_data.patent_upsert_job('/path/to/processed/data')
#     );
    
#     -- To set up automatic monitoring with a Databricks job:
#     -- 1. Create a notebook with the update logic
#     -- 2. Schedule it to run on your preferred interval
#     -- 3. Use the control table to track processed data and avoid duplicates
#     """
#     return instructions

# def incremental_load_instructions():
#     """
#     Generate instructions for incremental loading
#     """
#     instructions = """
#     -- Example notebook code for incremental loading:
    
#     # Check the last processed timestamp
#     last_process_df = spark.sql('''
#         SELECT MAX(update_timestamp) AS last_update 
#         FROM patent_data.patent_update_control
#     ''')
    
#     last_timestamp = last_process_df.first().last_update
    
#     if last_timestamp is None:
#         last_timestamp = '2000-01-01 00:00:00'
    
#     # Look for new files to process
#     new_files = dbutils.fs.ls('/path/to/incoming/data')
    
#     # Filter for files newer than last processed
#     # Note: This depends on your specific file structure
#     # You'll need to adapt this to your actual data organization
    
#     # If new files exist, process them
#     if len(new_files) > 0:
#         # Process new files and get path of processed data
#         processed_path = '/path/to/processed/output'
        
#         # Update all tables with the new data
#         result = spark.sql(f"SELECT patent_data.patent_upsert_job('{processed_path}')")
#         result_json = result.first()[0]
        
#         # Record the update in the control table
#         spark.sql(f'''
#             INSERT INTO patent_data.patent_update_control
#             VALUES (
#                 uuid(),
#                 current_timestamp(),
#                 '{processed_path}',
#                 '{result_json}'
#             )
#         ''')
#     """
#     return instructions

# def main():
#     """Main function to setup all tables and upsert mechanisms"""
    
#     spark = initialize_spark()
    
#     # Create the database
#     database_name = "patent_data"
#     spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    
#     # Create the control table for monitoring updates
#     create_control_table(spark, database_name)
    
#     # Base path to gold Delta tables
#     gold_base_path = "/Volumes/nokia-assginment-catalog/gold"
    
#     # Register all tables
#     print("Registering patent Delta tables...")
#     results = register_all_patent_tables(
#         gold_base_path=gold_base_path,
#         database_name=database_name,
#         drop_existing=False  # Set to True if you want to drop and recreate tables
#     )
    
#     # Print registration results
#     for table, status in results.items():
#         print(f"Table {table}: {status}")
    
#     # Create upsert functions instead of procedures
#     create_upsert_function(spark, database_name)
#     batch_upsert_function(spark, database_name)
    
#     # Print usage instructions
#     print("\nTo upsert data into tables, use the following functions:")
#     print(setup_upsert_usage_instructions())
    
#     print("\nTo set up incremental loading, here's a template:")
#     print(incremental_load_instructions())
    
#     print("\nSetup complete!")



In [0]:
# main()

In [0]:
# from pyspark.sql import SparkSession
# from delta.tables import DeltaTable
# import traceback
# import json
# import uuid

# def initialize_spark():
#     """Initialize Spark session with Delta Lake support"""
#     return SparkSession.builder \
#         .appName("Patent Delta Tables Registration") \
#         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#         .getOrCreate()

# def check_path_exists(path):
#     """Check if a path exists and is accessible"""
#     try:
#         dbutils.fs.ls(path)
#         return True
#     except:
#         return False

# def is_delta_table(spark, path):
#     """Check if a path is a valid Delta table"""
#     try:
#         DeltaTable.forPath(spark, path)
#         return True
#     except:
#         return False

# def register_and_upsert_table(spark, delta_path, table_name, database_name="patent_data"):
#     """
#     Register a Delta table if it doesn't exist and upsert data
    
#     Args:
#         spark: SparkSession
#         delta_path: Path to Delta table
#         table_name: Name of the table to create/update
#         database_name: Database to create the table in
    
#     Returns:
#         dict: Operation result
#     """
#     try:
#         # Create database if it doesn't exist
#         spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        
#         # Fully qualified table name
#         full_table_name = f"{database_name}.{table_name}"
        
#         # Read the Delta data (source)
#         source_df = spark.read.format("delta").load(delta_path)
        
#         # Check if source has data
#         source_count = source_df.count()
#         if source_count == 0:
#             return {
#                 "status": "warning",
#                 "message": f"Source Delta file at {delta_path} is empty"
#             }
            
#         # Check if table already exists
#         table_exists = False
#         existing_tables = [t.name for t in spark.catalog.listTables(database_name)]
#         if table_name in existing_tables:
#             table_exists = True
            
#         # For individual tables, use overwrite mode instead of merge
#         is_individual_table = "_individual" in table_name
        
#         if not table_exists:
#             # Create table if it doesn't exist
#             print(f"Creating table {full_table_name} from {delta_path}")
#             source_df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
            
#             # Configure table properties for optimization
#             spark.sql(f"""
#                 ALTER TABLE {full_table_name}
#                 SET TBLPROPERTIES (
#                     'delta.autoOptimize.optimizeWrite' = 'true',
#                     'delta.autoOptimize.autoCompact' = 'true',
#                     'delta.enableChangeDataFeed' = 'true',
#                     'delta.logRetentionDuration' = '30 days',
#                     'delta.tuneFileSizesForRewrites' = 'true'
#                 )
#             """)
            
#             print(f"Table {full_table_name} created successfully with {source_count} rows")
#             return {
#                 "status": "created",
#                 "message": f"Table {full_table_name} created with {source_count} rows"
#             }
#         elif is_individual_table:
#             # For individual tables, use complete replacement instead of merge
#             print(f"Replacing data in {full_table_name} from {delta_path} (individual table)")
#             source_df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
#             print(f"Data in {full_table_name} replaced successfully with {source_count} rows")
#             return {
#                 "status": "replaced",
#                 "message": f"Table {full_table_name} completely replaced with {source_count} rows"
#             }
#         else:
#             # For regular tables, perform upsert
#             if "publication_number" not in source_df.columns:
#                 return {
#                     "status": "warning",
#                     "message": f"Source data doesn't have publication_number column"
#                 }
                
#             print(f"Performing upsert to {full_table_name} from {delta_path}")
            
#             # Get target table
#             target_table = DeltaTable.forName(spark, full_table_name)
            
#             # Perform merge operation
#             target_table.alias("target").merge(
#                 source_df.alias("source"),
#                 "target.publication_number = source.publication_number"
#             ).whenMatchedUpdateAll(
#             ).whenNotMatchedInsertAll(
#             ).execute()
            
#             print(f"Upsert to {full_table_name} completed successfully")
#             return {
#                 "status": "updated",
#                 "message": f"Table {full_table_name} updated with upsert"
#             }
            
#     except Exception as e:
#         error_message = f"Error processing {table_name}: {str(e)}"
#         print(error_message)
#         print(traceback.format_exc())
#         return {
#             "status": "error",
#             "message": error_message
#         }

# def register_and_upsert_all_tables(gold_base_path="/Volumes/nokia-assginment-catalog/gold", 
#                                    database_name="patent_data"):
#     """
#     Process all patent gold tables - register if needed and upsert data
    
#     Args:
#         gold_base_path: Base path to gold Delta tables
#         database_name: Database to create tables in
        
#     Returns:
#         dict: Results for each table
#     """
#     spark = initialize_spark()
#     results = {}
    
#     # Create control table if it doesn't exist
#     try:
#         spark.sql(f"""
#         CREATE TABLE IF NOT EXISTS {database_name}.patent_update_control (
#             update_id STRING,
#             update_timestamp TIMESTAMP,
#             result STRING
#         ) USING DELTA
#         """)
#     except Exception as e:
#         print(f"Warning: Could not create control table: {str(e)}")
    
#     # Define tables to process
#     tables = [
#         {"path": f"{gold_base_path}/patents", "name": "patents"},
#         {"path": f"{gold_base_path}/ipc_classifications", "name": "ipc_classifications"},
#         {"path": f"{gold_base_path}/ipc_individual", "name": "ipc_individual"},
#         {"path": f"{gold_base_path}/inventors", "name": "inventors"},
#         {"path": f"{gold_base_path}/inventors_individual", "name": "inventors_individual"},
#         {"path": f"{gold_base_path}/applicants", "name": "applicants"},
#         {"path": f"{gold_base_path}/applicants_individual", "name": "applicants_individual"},
#         {"path": f"{gold_base_path}/us_applicants", "name": "us_applicants"},
#         {"path": f"{gold_base_path}/us_applicants_individual", "name": "us_applicants_individual"},
#         {"path": f"{gold_base_path}/claims", "name": "claims"},
#         {"path": f"{gold_base_path}/claims_individual", "name": "claims_individual"},
#         {"path": f"{gold_base_path}/complete_patents", "name": "complete_patents"}
#     ]
    
#     # Register and upsert each table
#     for table_info in tables:
#         delta_path = table_info["path"]
#         table_name = table_info["name"]
        
#         # Check if Delta path exists
#         if not check_path_exists(delta_path):
#             print(f"Delta path does not exist: {delta_path}")
#             results[table_name] = {"status": "skipped", "message": "Delta path not found"}
#             continue
            
#         # Check if it's a valid Delta table
#         if not is_delta_table(spark, delta_path):
#             print(f"Path is not a valid Delta table: {delta_path}")
#             results[table_name] = {"status": "skipped", "message": "Not a valid Delta table"}
#             continue
            
#         # Register and upsert the table
#         result = register_and_upsert_table(
#             spark, 
#             delta_path, 
#             table_name, 
#             database_name=database_name
#         )
        
#         results[table_name] = result
    
#     # Log this batch of updates in the control table
#     try:
#         result_json = json.dumps(results)
#         update_id = str(uuid.uuid4())
        
#         spark.sql(f"""
#         INSERT INTO {database_name}.patent_update_control
#         VALUES (
#             '{update_id}',
#             current_timestamp(),
#             '{result_json}'
#         )
#         """)
        
#         print(f"Update logged in control table with ID: {update_id}")
#     except Exception as e:
#         print(f"Warning: Could not log update in control table: {str(e)}")
    
#     return results

# def main():
#     """Main function that runs as part of the workflow after gold process notebook"""
    
#     # Base path to gold Delta tables
#     gold_base_path = "/Volumes/nokia-assginment-catalog/gold"
#     database_name = "patent_data"
    
#     print(f"Starting patent data registration and upsert process")
#     print(f"Source: {gold_base_path}")
#     print(f"Target database: {database_name}")
    
#     # Register and upsert all tables
#     results = register_and_upsert_all_tables(
#         gold_base_path=gold_base_path,
#         database_name=database_name
#     )
    
#     # Print summary
#     created_count = sum(1 for r in results.values() if r.get('status') == 'created')
#     updated_count = sum(1 for r in results.values() if r.get('status') == 'updated')
#     replaced_count = sum(1 for r in results.values() if r.get('status') == 'replaced')
#     error_count = sum(1 for r in results.values() if r.get('status') == 'error')
#     skipped_count = sum(1 for r in results.values() if r.get('status') == 'skipped')
#     warning_count = sum(1 for r in results.values() if r.get('status') == 'warning')
    
#     print("\n=== PROCESSING SUMMARY ===")
#     print(f"Tables created: {created_count}")
#     print(f"Tables updated via upsert: {updated_count}")
#     print(f"Tables completely replaced: {replaced_count}")
#     print(f"Tables with errors: {error_count}")
#     print(f"Tables skipped: {skipped_count}")
#     print(f"Tables with warnings: {warning_count}")
    
#     # Print details of any errors
#     if error_count > 0:
#         print("\n=== ERROR DETAILS ===")
#         for table, result in results.items():
#             if result.get('status') == 'error':
#                 print(f"{table}: {result.get('message')}")
    
#     print("\nProcess completed")

# main()

In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import traceback
import json
import uuid

def initialize_spark():
    """Initialize Spark session with Delta Lake support"""
    return SparkSession.builder \
        .appName("Patent Delta Tables Registration") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

def check_path_exists(path):
    """Check if a path exists and is accessible"""
    try:
        dbutils.fs.ls(path)
        return True
    except:
        return False

def is_delta_table(spark, path):
    """Check if a path is a valid Delta table"""
    try:
        DeltaTable.forPath(spark, path)
        return True
    except:
        return False

def check_new_data_processed():
    """
    Check if new data was processed in the previous gold layer step
    
    Returns:
        bool: Whether new data was processed
    """
    try:
        # Method 1: Check processing metadata table
        spark = initialize_spark()
        
        # Check if the processing_metadata table exists
        database_name = "patent_data"
        try:
            metadata_df = spark.sql(f"SELECT * FROM {database_name}.processing_metadata ORDER BY processing_timestamp DESC LIMIT 1")
            if metadata_df.count() > 0:
                last_record = metadata_df.first()
                new_data_processed = last_record.new_data_processed
                print(f"Found processing metadata: new_data_processed={new_data_processed}")
                return new_data_processed
        except:
            print("Could not query processing_metadata table")
        
        # Method 2: Check processing status Delta file
        try:
            processing_status_path = "/Volumes/nokia-assginment-catalog/processing_status/status"
            if check_path_exists(processing_status_path):
                status_df = spark.read.format("delta").load(processing_status_path)
                if status_df.count() > 0:
                    status = status_df.first().new_data_processed
                    print(f"Found processing status file: new_data_processed={status}")
                    return status
        except:
            print("Could not read processing status file")
        
        # Method 3: Check for previous notebook result
        try:
            # Try to get result from previous notebook
            prev_result_str = dbutils.notebook.entry_point.getDbutils().notebook().getContext().parentContext().get("result")
            if prev_result_str:
                prev_result = json.loads(prev_result_str)
                if "new_data_processed" in prev_result:
                    status = prev_result["new_data_processed"]
                    print(f"Found previous notebook result: new_data_processed={status}")
                    return status
        except:
            print("Could not get previous notebook result")
        
        # Default: If we can't determine, assume there might be new data
        print("Could not determine if new data was processed, assuming yes")
        return True
        
    except Exception as e:
        print(f"Error checking for new data: {str(e)}")
        print(traceback.format_exc())
        # If there's an error, assume there might be new data
        return True

def register_and_upsert_table(spark, delta_path, table_name, database_name="patent_data"):
    """
    Register a Delta table if it doesn't exist and upsert data
    
    Args:
        spark: SparkSession
        delta_path: Path to Delta table
        table_name: Name of the table to create/update
        database_name: Database to create the table in
    
    Returns:
        dict: Operation result
    """
    try:
        # Debug - what's in the gold delta path?
        print(f"Contents of {delta_path}:")
        path_files = dbutils.fs.ls(delta_path)
        for file in path_files[:5]:  # Show first 5 items
            print(f"  - {file.name} ({file.size} bytes)")
        
        # Debug - how many batches are in silver layer
        try:
            silver_path = "/Volumes/nokia-assginment-catalog/silver/patent_data"
            silver_dirs = [d for d in dbutils.fs.ls(silver_path) if d.isDir() and not d.name.startswith('_')]
            print(f"Silver layer has {len(silver_dirs)} batch directories")
        except:
            print("Could not access silver layer to count batches")
        
        # Debug - how many batches have checkpoints
        try:
            checkpoint_location = "/Volumes/nokia-assginment-catalog/checkpoints/checkpoints_data/gold_autoloader/"
            if check_path_exists(checkpoint_location):
                checkpoints = [d for d in dbutils.fs.ls(checkpoint_location) if not d.name.startswith('_')]
                print(f"Found {len(checkpoints)} checkpoint files")
        except:
            print("Could not check checkpoint files")

        # Create database if it doesn't exist
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        
        # Fully qualified table name
        full_table_name = f"{database_name}.{table_name}"
        
        # Read the Delta data (source)
        source_df = spark.read.format("delta").load(delta_path)
        
        # Check if source has data and print more details for debugging
        source_count = source_df.count()
        print(f"Source Delta file at {delta_path} has {source_count} records")
        
        if source_count == 0:
            return {
                "status": "warning",
                "message": f"Source Delta file at {delta_path} is empty"
            }
            
        # Check if table already exists
        table_exists = False
        existing_tables = [t.name for t in spark.catalog.listTables(database_name)]
        if table_name in existing_tables:
            table_exists = True
            
        # For individual tables, use overwrite mode instead of merge
        is_individual_table = "_individual" in table_name
        
        if not table_exists:
            # Create table if it doesn't exist
            print(f"Creating table {full_table_name} from {delta_path}")
            source_df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
            
            # Configure table properties for optimization
            spark.sql(f"""
                ALTER TABLE {full_table_name}
                SET TBLPROPERTIES (
                    'delta.autoOptimize.optimizeWrite' = 'true',
                    'delta.autoOptimize.autoCompact' = 'true',
                    'delta.enableChangeDataFeed' = 'true',
                    'delta.logRetentionDuration' = '30 days',
                    'delta.tuneFileSizesForRewrites' = 'true'
                )
            """)
            
            print(f"Table {full_table_name} created successfully with {source_count} rows")
            return {
                "status": "created",
                "message": f"Table {full_table_name} created with {source_count} rows"
            }
        elif is_individual_table:
            # For individual tables, keep a record of how many rows we're about to replace
            target_df = spark.read.table(full_table_name)
            target_count = target_df.count()
            print(f"Replacing {target_count} existing rows with {source_count} rows in {full_table_name}")
            
            # If source has fewer records than target, this could indicate a problem
            if source_count < target_count and target_count > 0:
                print(f"WARNING: Source has fewer records ({source_count}) than existing table ({target_count})")
            
            # For individual tables, use complete replacement instead of merge
            source_df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
            print(f"Data in {full_table_name} replaced successfully with {source_count} rows")
            return {
                "status": "replaced",
                "message": f"Table {full_table_name} completely replaced with {source_count} rows (previous: {target_count})"
            }
        else:
            # For regular tables, perform upsert
            if "publication_number" not in source_df.columns:
                return {
                    "status": "warning",
                    "message": f"Source data doesn't have publication_number column"
                }
            
            # Get target table and count before merge
            target_table = DeltaTable.forName(spark, full_table_name)
            target_count_before = spark.read.table(full_table_name).count()
            
            print(f"Performing upsert to {full_table_name} from {delta_path}")
            print(f"Target table has {target_count_before} rows before merge")
            
            # Sample a few publication numbers from source for debugging
            sample_pubs = source_df.select("publication_number").limit(5).collect()
            print(f"Sample publication numbers from source: {[row.publication_number for row in sample_pubs]}")
            
            # Perform merge operation
            target_table.alias("target").merge(
                source_df.alias("source"),
                "target.publication_number = source.publication_number"
            ).whenMatchedUpdateAll(
            ).whenNotMatchedInsertAll(
            ).execute()
            
            # Count after merge to see if anything changed
            target_count_after = spark.read.table(full_table_name).count()
            rows_changed = target_count_after - target_count_before
            
            print(f"Upsert to {full_table_name} completed. Rows before: {target_count_before}, after: {target_count_after}")
            print(f"Net change in rows: {rows_changed}")
            
            return {
                "status": "updated",
                "message": f"Table {full_table_name} updated with upsert. Net new rows: {rows_changed}"
            }
            
    except Exception as e:
        error_message = f"Error processing {table_name}: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        return {
            "status": "error",
            "message": error_message
        }

def register_and_upsert_all_tables(gold_base_path="/Volumes/nokia-assginment-catalog/gold", 
                                   database_name="patent_data"):
    """
    Process all patent gold tables - register if needed and upsert data
    
    Args:
        gold_base_path: Base path to gold Delta tables
        database_name: Database to create tables in
        
    Returns:
        dict: Results for each table
    """
    spark = initialize_spark()
    results = {}
    
    # Create control table if it doesn't exist
    try:
        spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {database_name}.patent_update_control (
            update_id STRING,
            update_timestamp TIMESTAMP,
            result STRING
        ) USING DELTA
        """)
    except Exception as e:
        print(f"Warning: Could not create control table: {str(e)}")
    
    # Define tables to process
    tables = [
        {"path": f"{gold_base_path}/patents", "name": "patents"},
        {"path": f"{gold_base_path}/ipc_classifications", "name": "ipc_classifications"},
        {"path": f"{gold_base_path}/ipc_individual", "name": "ipc_individual"},
        {"path": f"{gold_base_path}/inventors", "name": "inventors"},
        {"path": f"{gold_base_path}/inventors_individual", "name": "inventors_individual"},
        {"path": f"{gold_base_path}/applicants", "name": "applicants"},
        {"path": f"{gold_base_path}/applicants_individual", "name": "applicants_individual"},
        {"path": f"{gold_base_path}/us_applicants", "name": "us_applicants"},
        {"path": f"{gold_base_path}/us_applicants_individual", "name": "us_applicants_individual"},
        {"path": f"{gold_base_path}/claims", "name": "claims"},
        {"path": f"{gold_base_path}/claims_individual", "name": "claims_individual"},
        {"path": f"{gold_base_path}/complete_patents", "name": "complete_patents"}
    ]
    
    # Register and upsert each table
    for table_info in tables:
        delta_path = table_info["path"]
        table_name = table_info["name"]
        
        # Check if Delta path exists
        if not check_path_exists(delta_path):
            print(f"Delta path does not exist: {delta_path}")
            results[table_name] = {"status": "skipped", "message": "Delta path not found"}
            continue
            
        # Check if it's a valid Delta table
        if not is_delta_table(spark, delta_path):
            print(f"Path is not a valid Delta table: {delta_path}")
            results[table_name] = {"status": "skipped", "message": "Not a valid Delta table"}
            continue
            
        # Register and upsert the table
        result = register_and_upsert_table(
            spark, 
            delta_path, 
            table_name, 
            database_name=database_name
        )
        
        results[table_name] = result
    
    # Log this batch of updates in the control table
    try:
        result_json = json.dumps(results)
        update_id = str(uuid.uuid4())
        
        spark.sql(f"""
        INSERT INTO {database_name}.patent_update_control
        VALUES (
            '{update_id}',
            current_timestamp(),
            '{result_json}'
        )
        """)
        
        print(f"Update logged in control table with ID: {update_id}")
    except Exception as e:
        print(f"Warning: Could not log update in control table: {str(e)}")
    
    return results

def main():
    """Main function that runs as part of the workflow after gold process notebook"""
    
    # Check if we should run based on whether new data was processed in the previous step
    try:
        dbutils.widgets.dropdown("force_update", "false", ["true", "false"], "Force Table Update")
        force_update = dbutils.widgets.get("force_update") == "true"
    except:
        force_update = False
    
    # Check if new data was processed in the previous step
    new_data_processed = check_new_data_processed()
    
    # Skip processing if no new data and not forcing update
    if not new_data_processed and not force_update:
        print("No new data was processed and force_update=false, skipping table registration")
        result = {
            "status": "skipped",
            "message": "No new data was processed and force_update=false"
        }
        dbutils.notebook.exit(json.dumps(result))
        return
    
    # Base path to gold Delta tables
    gold_base_path = "/Volumes/nokia-assginment-catalog/gold"
    database_name = "patent_data"
    
    print(f"Starting patent data registration and upsert process")
    if new_data_processed:
        print("New data was processed in previous step")
    if force_update:
        print("Force update requested")
        
    print(f"Source: {gold_base_path}")
    print(f"Target database: {database_name}")
    
    # Register and upsert all tables
    results = register_and_upsert_all_tables(
        gold_base_path=gold_base_path,
        database_name=database_name
    )
    
    # Print summary
    created_count = sum(1 for r in results.values() if r.get('status') == 'created')
    updated_count = sum(1 for r in results.values() if r.get('status') == 'updated')
    replaced_count = sum(1 for r in results.values() if r.get('status') == 'replaced')
    error_count = sum(1 for r in results.values() if r.get('status') == 'error')
    skipped_count = sum(1 for r in results.values() if r.get('status') == 'skipped')
    warning_count = sum(1 for r in results.values() if r.get('status') == 'warning')
    
    print("\n=== PROCESSING SUMMARY ===")
    print(f"Tables created: {created_count}")
    print(f"Tables updated via upsert: {updated_count}")
    print(f"Tables completely replaced: {replaced_count}")
    print(f"Tables with errors: {error_count}")
    print(f"Tables skipped: {skipped_count}")
    print(f"Tables with warnings: {warning_count}")
    
    # Print details of any errors
    if error_count > 0:
        print("\n=== ERROR DETAILS ===")
        for table, result in results.items():
            if result.get('status') == 'error':
                print(f"{table}: {result.get('message')}")
    
    print("\nProcess completed")
    
    # Return results as JSON
    final_result = {
        "status": "completed",
        "created": created_count,
        "updated": updated_count,
        "replaced": replaced_count,
        "errors": error_count,
        "skipped": skipped_count,
        "warnings": warning_count
    }
    dbutils.notebook.exit(json.dumps(final_result))

main()